### Generate article with prompt

SDK/Packages

In [ ]:
!pip install --upgrade --quiet google-genai langgraph langchain \
langchain-google-genai langchain-community tavily-python pydantic

In [ ]:
import IPython
from IPython.display import Markdown, display, Image
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

import google.generativeai as genai
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
''' init gemini model and search tool '''
model = ChatGoogleGenerativeAI(model=ModelId, temperature=0)
tavily_tool = TavilySearchResults(max_results=5)

''' define templates and runnables '''
outline_template = ChatPromptTemplate.from_template(
    'Create a detailed outline for an essay on {topic}')

''' research web search '''
def research_fn(topic):
  response = tavily_tool.invoke({'query': topic})
  return "\n".join(f"- {result['content']}" for result in response)

writing_template = ChatPromptTemplate.from_template(
    'based on the outline and research, write a 3-paragraph essay on {topic}\
    :\n\nOutline:\n{outline}\n\nResearch:\n{research}\n\nEssay'
)
''' individual chains '''
outline_chain = LLMChain(llm=model, prompt=outline_template)
writing_chain = LLMChain(llm=model, prompt=writing_template)

''' pipeline operator to combine chains '''
chain = (
    outline_chain
    |(
        lambda result: {
            'topic': result['topic'],
            'outline': result['text'],
            'research': research_fn(result['topic']),
        }
    )
    | writing_chain
    | (lambda result: result['text'])
    | StrOutputParser()
)

Generate article

In [ ]:
article = chain.invoke({'topic': prompt})
article